## Example Notebook on getting data from STAC directly with GDAL

In [2]:
import os

Due to a Bug in GDAL, which did not accept local file:// URIS that terrabyte uses please make sure you have at minimum GDAL Version 3.8.0 installed. (If this is not available yet you can use the master branch installabe from the gdal-master conda channel 

In [1]:
!gdalinfo --version

GDAL 3.8.0dev-c95490018a-dirty, released 2023/07/21


GDAL directly uses the search API of STAC. The resulting data of a search query can be used as input for a GDAL command. 

In [14]:
# GDAL directly uses the search API of STAC
stac_search_url="https://stac.terrabyte.lrz.de/public/api/search"
# STAC Collection to search
collection="cop-dem-glo-90"
# Name of the Asset in the collection above we want do download
asset="data"

# Area of interest 
bboxMunich = [11.089325,47.893787,11.923599,48.329322 ]
#so we can switch bounding box easily
bbox=bboxMunich

#extra gdal option to pass. Here we activate Bigtiff support and LZW Compression as well as give a name for the Band in the Format ArcGIS expects
gdalOptions=f"-co COMPRESS=LZW -co BIGTIFF=YES -mo band_1={asset}"
outputFilename="gdal_COPDEM_example.tif"

In [13]:
# construct the gdal URL to access STAC search with our parameters
gdal_inputURl=f"STACIT:\"{stac_search_url}?collections={collection}&bbox={','.join(str(b) for b in bbox)}\":asset={asset}"
gdal_inputURl

'STACIT:"https://stac.terrabyte.lrz.de/public/api/search?collections=cop-dem-glo-90&bbox=11.089325,47.893787,11.923599,48.329322":asset=data'

Specifying the Bounding Box as as search Parameter to STAC only returns those results that intersect the bounding box

In [9]:
infoCmd=f"gdalinfo '{gdal_inputURl}'"
print(infoCmd)
#os.system(f"echo {cmdUrl}")


gdalinfo 'STACIT:"https://stac.terrabyte.lrz.de/public/api/search?collections=cop-dem-glo-30&bbox=11.089325,47.893787,11.923599,48.329322":asset=data'


In [10]:
os.system(infoCmd)

256

We specify the Bounding Box again to gdal itself to clip the output exactly to the bounding box. Be aware that gdal expects a different Order of Coordinates 

In [5]:
gdal_projwin=f"-projwin {' '.join(str(bbox[i]) for i in [0,3,2,1])}"
gdal_projwin                                  

'-projwin 11.089325 48.329322 11.923599 47.893787'

In [19]:
translateCmd=f"gdal_translate '{gdal_inputURl}' {outputFilename} {gdal_projwin} {gdalOptions}"
print(translateCmd)

gdal_translate 'STACIT:"https://stac.terrabyte.lrz.de/public/api/search?collections=cop-dem-glo-90&bbox=11.089325,47.893787,11.923599,48.329322":asset=data' gdal_COPDEM_example.tif -projwin 11.089325 48.329322 11.923599 47.893787 -co COMPRESS=LZW -co BIGTIFF=YES -mo band_1=data


In [20]:
os.system(translateCmd)

Input file size is 1201, 2401
0...10...20...30...40...50...60...70...80...90...100 - done.


0